In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
credit_card_data=pd.read_csv('fraudTrain.csv')

In [3]:
#Hence no  null values are present

In [4]:
credit_card_data['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [5]:
# Since trans_date_trans_time, merchant, cateogry,first,last,gender,street,city,state,job,dob,trans_num

In [6]:
#since trans_data_trans_time is non cateogorical it has to be dropped

In [7]:
credit_card_data=credit_card_data.drop('trans_date_trans_time', axis=1)

In [8]:
credit_card_data=credit_card_data.drop(columns=['merchant','first','last','street'],axis=1)

In [9]:
credit_card_data['gender'].value_counts()

F    709863
M    586812
Name: gender, dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [11]:
credit_card_data['gender']=le.fit_transform(credit_card_data['gender'])

In [12]:
#city paramter has large class values so it has to be dropped.

In [13]:
credit_card_data=credit_card_data.drop('city',axis=1)

In [14]:
credit_card_data['category']=le.fit_transform(credit_card_data['category'])

In [15]:
credit_card_data['job'].value_counts()

Film/video editor             9779
Exhibition designer           9199
Naval architect               8684
Surveyor, land/geomatics      8680
Materials engineer            8270
                              ... 
Information officer              8
Veterinary surgeon               8
Ship broker                      7
Contracting civil engineer       7
Warehouse manager                7
Name: job, Length: 494, dtype: int64

In [16]:
credit_card_data['dob']

0          1988-03-09
1          1978-06-21
2          1962-01-19
3          1967-01-12
4          1986-03-28
              ...    
1296670    1961-11-24
1296671    1979-12-11
1296672    1967-08-30
1296673    1980-08-18
1296674    1995-08-16
Name: dob, Length: 1296675, dtype: object

In [17]:
credit_card_data['trans_num']

0          0b242abb623afc578575680df30655b9
1          1f76529f8574734946361c461b024d99
2          a1a22d70485983eac12b5b88dad1cf95
3          6b849c168bdad6f867558c3793159a81
4          a41d7549acf90789359a9aa5346dcb46
                         ...               
1296670    440b587732da4dc1a6395aba5fb41669
1296671    278000d2e0d2277d1de2f890067dcc0a
1296672    483f52fe67fabef353d552c1e662974c
1296673    d667cdcbadaaed3da3f4020e83591c83
1296674    8f7c8e4ab7f25875d753b422917c98c9
Name: trans_num, Length: 1296675, dtype: object

In [18]:
credit_card_data=credit_card_data.drop(columns=['trans_num','dob','job','state'],axis=1)

In [19]:
credit_card_data['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [20]:
#Since the data is not balanced we will take a random sample of class 0 from the large length

In [21]:
legit=credit_card_data[credit_card_data.is_fraud==0]
fraud= credit_card_data[credit_card_data.is_fraud==1]

In [22]:
legit=legit.sample(n=7506)

In [23]:
new_dataset= pd.concat([legit,fraud], axis=0)

#Checking the count for the newly formed dataset
new_dataset['is_fraud'].value_counts()

0    7506
1    7506
Name: is_fraud, dtype: int64

In [24]:
X=new_dataset.drop('is_fraud',axis=1)

In [25]:
Y=new_dataset['is_fraud']

In [26]:
#TEST TRAIN SPLITTING ON DATA

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
  X,Y, random_state=2,test_size=0.2, shuffle=True)

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rs=RandomForestClassifier(n_estimators=50)

In [31]:
rs.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50)

In [32]:
import pickle

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(rs, f)



In [33]:
pred=rs.predict(X_test)

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
print(accuracy_score(y_test,pred))

0.9587079587079587


In [36]:
#SINCE OUR MODEL  IS performing good now we will find the prediction for test file

In [37]:
test_data=pd.read_csv('fraudTest.csv')

In [38]:
test_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [39]:
test_data.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [40]:
test_data=test_data.drop(columns=['trans_num','dob','job','state','merchant','first','last','street','trans_date_trans_time','city'],axis=1)

In [41]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  555719 non-null  int64  
 1   cc_num      555719 non-null  int64  
 2   category    555719 non-null  object 
 3   amt         555719 non-null  float64
 4   gender      555719 non-null  object 
 5   zip         555719 non-null  int64  
 6   lat         555719 non-null  float64
 7   long        555719 non-null  float64
 8   city_pop    555719 non-null  int64  
 9   unix_time   555719 non-null  int64  
 10  merch_lat   555719 non-null  float64
 11  merch_long  555719 non-null  float64
 12  is_fraud    555719 non-null  int64  
dtypes: float64(5), int64(6), object(2)
memory usage: 55.1+ MB


In [42]:
test_data['category']=le.fit_transform(test_data['category'])
test_data['gender']=le.fit_transform(test_data['gender'])

In [43]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  555719 non-null  int64  
 1   cc_num      555719 non-null  int64  
 2   category    555719 non-null  int32  
 3   amt         555719 non-null  float64
 4   gender      555719 non-null  int32  
 5   zip         555719 non-null  int64  
 6   lat         555719 non-null  float64
 7   long        555719 non-null  float64
 8   city_pop    555719 non-null  int64  
 9   unix_time   555719 non-null  int64  
 10  merch_lat   555719 non-null  float64
 11  merch_long  555719 non-null  float64
 12  is_fraud    555719 non-null  int64  
dtypes: float64(5), int32(2), int64(6)
memory usage: 50.9 MB


In [44]:
X=test_data.drop('is_fraud',axis=1)

In [45]:
Y=test_data['is_fraud']

In [46]:
pred_test=rs.predict(X)

In [47]:
accuracy_score(Y,pred_test)

0.9725184850616948

In [48]:
X_train.shape

(12009, 12)

In [51]:
X_train.columns

Index(['Unnamed: 0', 'cc_num', 'category', 'amt', 'gender', 'zip', 'lat',
       'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long'],
      dtype='object')